In [ ]:
import numpy as np
import inv_utils as iu
import me_utils as meu
from helita.io import lp
from einops import rearrange
import hmi_plot as hp
from uncertainties import  unumpy
import importlib
import warnings
# Suppress the specific warning
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float32'> type is zero")
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float64'> type is zero")

In [ ]:

importlib.reload(iu)
importlib.reload(meu)
print('reloaded') 

In [ ]:
# Load the configuration from the JSON file
input_config = iu.load_yaml_config('input_config.yaml')

In [ ]:
# Check the input configuration
config = iu.check_input_config(input_config, pprint=True, confirm=False)

In [ ]:

# Extract the input parameters
data_dir = config['data_dir']
save_dir = config['save_dir']
crisp_im = config['crisp_im']
xorg = config['xorg']
xsize = config['xsize']
yorg = config['yorg']
ysize = config['ysize']
xrange = config['xrange']
yrange = config['yrange']
tt = config['time_index']
scale = config['scale']
is_north_up = config['is_north_up']
crop = config['crop']
shape = config['shape']
best_frame = config['best_frame']
contrasts = config['contrasts']
hmi_con_series = config['hmi_con_series']
hmi_mag_series = config['hmi_mag_series']
email = config['email']
fov_angle = config['fov_angle']
plot_sst_pointings_flag = config['plot_sst_pointings_flag']
plot_hmi_ic_mag_flag = config['plot_hmi_ic_mag_flag']
plot_crisp_image_flag = config['plot_crisp_image_flag']
verbose = config['verbose']

In [ ]:
# Extract the fits information from the header
fits_info = config['fits_info']
nx = fits_info['nx']
ny = fits_info['ny']
mu = fits_info['mu']
x1 = fits_info['hplnt'][tt][0]
x2 = fits_info['hplnt'][tt][1]
y1 = fits_info['hpltt'][tt][0]
y2 = fits_info['hpltt'][tt][1]
tobs = fits_info['all_start_times'][tt]
tstart = fits_info['start_time_obs']
tend = fits_info['end_time_obs']
hplnt = fits_info['hplnt']
hpltt = fits_info['hpltt']
all_start_times = fits_info['all_start_times']
central_wavelength = fits_info['central_wavelength']

In [ ]:
# Reset the x and y ranges if cropping is enabled
if crop:
    x_list = np.linspace(x1, x2, num=nx)
    y_list = np.linspace(y1, y2, num=ny)
    x_list = x_list[xrange[0]:xrange[1]]
    y_list = y_list[yrange[0]:yrange[1]]
    x1 = x_list[0]
    x2 = x_list[-1]
    y1 = y_list[0]
    y2 = y_list[-1]
    nx = xsize
    ny = ysize

In [ ]:
if plot_sst_pointings_flag:
    hp.plot_sst_pointings(tstart, hmi_con_series, hplnt, hpltt,figsize=(6, 6), email=email, save_dir=save_dir)

In [ ]:
if plot_hmi_ic_mag_flag:
    hp.plot_hmi_ic_mag(tobs, hmi_con_series, hmi_mag_series, email, x1, x2, y1, y2, save_dir=save_dir, figsize=(10, 5),  is_north_up=is_north_up, fov_angle=fov_angle, shape=shape)

In [ ]:
if plot_crisp_image_flag:
    print('SST CRISP image with North up:', not(is_north_up))
    iu.plot_crisp_image(crisp_im, tt=tt, ss=0, ww=0, figsize=(6,6), fontsize=10, rot_fov=fov_angle, north_up=not(is_north_up), crop=crop, xrange=xrange, yrange=yrange, xtick_range=[x1,x2], ytick_range=[y1,y2])

In [ ]:
inversion_config = iu.load_yaml_config('inversion_config.yaml')
# Load the variables from the inversion configuration
dtype = inversion_config['dtype']
nthreads = inversion_config['nthreads']
sigma_strength= inversion_config['sigma_strength']
sigma_list = inversion_config['sigma_list']
erh = inversion_config['erh']
init_model_params = inversion_config['init_model_params']
nRandom1 = inversion_config['nRandom1']
nIter1 = inversion_config['nIter1']
chi2_thres1 = inversion_config['chi2_thres1']
median_filter_chi2_mean_thres = inversion_config['median_filter_chi2_mean_thres']
median_filter_size = inversion_config['median_filter_size']
nRandom2 = inversion_config['nRandom2']
nIter2 = inversion_config['nIter2']
chi2_thres2 = inversion_config['chi2_thres2']
nIter3 = inversion_config['nIter3']
chi2_thres3 = inversion_config['chi2_thres3']
alpha_strength = inversion_config['alpha_strength']
alpha_list = inversion_config['alpha_list']
nan_mask_replacements = inversion_config['nan_mask_replacements']


In [ ]:
ll = meu.load_crisp_frame(crisp_im, tt, crop=crop, xrange=xrange, yrange=yrange)

In [ ]:
obs, sig, l0, me = meu.init_me_model(ll, sigma_strength, sigma_list, erh=erh, dtype=dtype, nthreads=nthreads)

In [ ]:
Imodel = meu.init_model(me, ny, nx, init_model_params=init_model_params, dtype=dtype)

In [ ]:
Imodel, syn, chi2 = meu.run_randomised_me_inversion(Imodel, me, obs, sig, nRandom=nRandom1, nIter=nIter1, chi2_thres=chi2_thres1, mu=mu, verbose=verbose)
masked_chi2_mean = iu.masked_mean(chi2, ll.mask)
if verbose:
    print(f'Masked chi2 mean: {masked_chi2_mean:.2f}')
    iu.plot_inversion_output(Imodel, ll.mask, scale=scale, save_fig=False)
    iu.plot_mag(Imodel, ll.mask, scale=scale, save_fig=False)

In [ ]:
Imodel = meu.apply_median_filter_based_on_chi2(Imodel, masked_chi2_mean, median_filter_chi2_mean_thres, median_filter_size)
if verbose:    
    iu.plot_inversion_output(Imodel,ll.mask,scale=scale, save_fig=False)
    iu.plot_mag(Imodel,ll.mask,scale=scale, save_fig=False)

In [ ]:
Imodel, syn, chi2 = meu.run_randomised_me_inversion(Imodel, me, obs, sig, nRandom=nRandom2, nIter=nIter2, chi2_thres=chi2_thres2, mu=mu, verbose=verbose)
masked_chi2_mean = iu.masked_mean(chi2, ll.mask)
if verbose:
    print(f'Masked chi2 mean: {masked_chi2_mean:.2f}')
    iu.plot_inversion_output(Imodel, ll.mask, scale=scale, save_fig=False)
    iu.plot_mag(Imodel, ll.mask, scale=scale, save_fig=False)

In [ ]:
mo, syn, chi2 = meu.run_spatially_regularized_inversion(me, Imodel, obs, sig, nIter3, chi2_thres3, mu, alpha_strength, alpha_list, method=1, delay_bracket=3, dtype=dtype,verbose=True)
errors = me.estimate_uncertainties(np.squeeze(mo), obs, sig, mu=mu)
corrected_mo = meu.correct_velocities_for_cavity_error(mo, ll.cmap, l0, global_offset=0.0)
if verbose:
    print(f'Masked chi2 mean: {masked_chi2_mean:.2f}')
    iu.plot_inversion_output(corrected_mo,ll.mask,scale=scale, save_fig=False)
    iu.plot_mag(corrected_mo,ll.mask,scale=scale, save_fig=False)

In [ ]:
masked_model= meu.apply_mask_to_model(corrected_mo, ll.mask, nan_mask_replacements)
masked_errors = meu.apply_mask_to_model(errors, ll.mask, nan_mask_replacements)
if verbose:
    print(f'Masked chi2 mean: {masked_chi2_mean:.2f}')
    iu.plot_inversion_output(masked_model,mask=None,scale=scale, save_fig=False)
    iu.plot_inversion_output(masked_errors,mask=None,scale=scale, save_fig=False, apply_median_filter=True, filter_index=[1,2], filter_size=3)
    iu.plot_mag(masked_model,mask=None,scale=scale, save_fig=False)

---

In [ ]:
# iu.plot_sst_blos_bhor(blos_cube, bhor_cube, tt=tt,xrange=xrange, yrange=yrange, figsize=(20,10), fontsize=12, crop=crop)

In [ ]:
importlib.reload(iu)
importlib.reload(meu)

In [ ]:

model_im = rearrange(masked_model, 'ny nx nparams -> nparams ny nx')
errors_im = rearrange(masked_errors, 'ny nx nparams -> nparams ny nx')

In [ ]:
# Create arrays with uncertainties
B_with_errors = unumpy.uarray(model_im[0], errors_im[0])
inc_with_errors = unumpy.uarray(model_im[1], errors_im[1])

# Calculate Blos and Bhor with propagated errors
Blos_with_errors = B_with_errors * unumpy.cos(inc_with_errors)
Bhor_with_errors = B_with_errors * unumpy.sin(inc_with_errors)

# Extract nominal values and standard deviations
Blos = unumpy.nominal_values(Blos_with_errors)
Blos_err = unumpy.std_devs(Blos_with_errors)

Bhor = unumpy.nominal_values(Bhor_with_errors)
Bhor_err = unumpy.std_devs(Bhor_with_errors)


In [ ]:
Bhor_err_clipped = np.clip(Bhor_err, a_min=0, a_max=np.max(Bhor))
Blos_err_clipped = np.clip(Blos_err, a_min=0, a_max=np.max(np.abs(Blos)))

In [ ]:

# Extend model_im by adding Blos and Bhor and 10 and 11 indices
model_im = np.concatenate((model_im, Blos[np.newaxis], Bhor[np.newaxis]), axis=0)
errors_im = np.concatenate((errors_im, Blos_err_clipped[np.newaxis], Bhor_err_clipped[np.newaxis]), axis=0)

In [ ]:

idl_model_im = rearrange(model_im, 'nparams ny nx -> nparams nx ny')
idl_errors_im = rearrange(errors_im, 'nparams ny nx -> nparams nx ny')

In [ ]:
idl_model_im.shape

In [ ]:
# combine input_config and inversion_config dictionaries
full_config = {**input_config, **inversion_config}

In [ ]:
fits_header = iu.load_fits_header(crisp_im, out_dict=False)

In [ ]:
inversion_out_list = ["Bstr", "Binc", "Bazi", "Vlos", "Vdop", "etal", "damp", "S0", "S1", "Blos", "Bhor"]
inverstion_error_out_list = ["Bstr_err", "Binc_err", "Bazi_err", "Vlos_err", "Vdop_err", "etal_err", "damp_err", "S0_err", "S1_err", "Blos_err", "Bhor_err"]

In [ ]:
inversion_save_fits_list = ["Bstr", "Binc", "Bazi", "Vlos", "Blos", "Bhor"]
inversion_save_errors = True 

In [ ]:
time_string = all_start_times[tt].replace(':', '').replace(' ', '_T')
for var in inversion_save_fits_list:
    var_index = inversion_out_list.index(var)
    out_file_name = save_dir + f'{inversion_out_list[var_index]}_{str(int(central_wavelength))}_{time_string}.fits'
    iu.save_fits(idl_model_im[var_index], fits_header, out_file_name, overwrite=True)
    if inversion_save_errors:
        out_file_name = save_dir + f'{inversion_out_list[var_index]}_err_{str(int(central_wavelength))}_{time_string}.fits'
        iu.save_fits(idl_errors_im[var_index], fits_header, out_file_name, overwrite=True)

In [ ]:
import numpy as np
import struct

def make_lp_header(array, nt=None, t_start=None, delta_t=None):
    """
    Create a standard header for saving La Palma data.
    
    Parameters:
        array (numpy.ndarray): The data array.
        nt (int, optional): Number of time steps.
        t_start (str, optional): Start time.
        delta_t (str, optional): Time interval.
    
    Returns:
        str: The header string.
    """
    # Check image dimensions
    sZ = array.shape
    dims = len(sZ)
    
    if dims < 2:
        raise ValueError("Only 2D or 3D files are supported")
    
    datatype_map = {
        np.uint8: 1,
        np.int16: 2,
        np.int32: 3,
        np.float32: 4
    }
    
    datatype = datatype_map.get(array.dtype.type, 4)
    if nt is not None:
        dims = 3
    if dims == 2:
        ny, nx = sZ
        nt = 1
    else:
        nt, ny, nx = sZ
    
    header = f"datatype={datatype}, dims={dims}, nx={nx}, ny={ny}, nt={nt}"
    
    endianstr = 'l' if struct.unpack('<I', struct.pack('=I', 1))[0] == 1 else 'b'
    header += f", endian={endianstr}"
    
    if t_start is not None:
        header += f", t_start={t_start}"
    if delta_t is not None:
        header += f", delta_t={delta_t}" if isinstance(delta_t, str) else f", delta_t={str(delta_t)}"
    
    return header

def lp_write(image, filename, extraheader=''):
    """
    Writes La Palma data to a file with a specified header.
    
    Parameters:
        image (numpy.ndarray): The image data to write.
        filename (str): The file to write the data to.
        extraheader (str, optional): Additional header information.
    """
    # Check image dimensions
    sZ = image.shape
    dims = len(sZ)
    
    if dims < 2:
        raise ValueError("Only 2D or 3D files are supported")
    
    if dims == 2:
        # Convert 2D array to 3D array with 1 timestep
        image = image[np.newaxis, :, :]
        sZ = image.shape
        dims = len(sZ)
    
    if dims == 3:
        # Reorder dimensions to (nt, nx, ny)
        image = image.transpose((0, 2, 1))
        sZ = image.shape
    
    nt, nx, ny = sZ
    datatype = image.dtype
    
    # Create the header
    bheader = make_lp_header(image, nt)
    header = extraheader + ' : ' + bheader
    
    with open(filename, 'wb') as f:
        # Write the header (first 512 bytes)
        header_bytes = header.encode('ascii')
        header_bytes_padded = header_bytes.ljust(512, b'\x00')
        f.write(header_bytes_padded)
        
        # Convert image to float32 if the datatype is not supported
        if datatype not in [np.uint8, np.int16, np.int32, np.float32]:
            image = image.astype(np.float32)
            datatype = image.dtype

        # Write the image data
        f.write(image.tobytes())

# # Example usage
# image = np.random.rand(10, 100, 100).astype(np.float32)  # Example 3D image (nt, ny, nx)
# filename = 'example.lp'
# extraheader = 'Additional Header Information'
# lp_write(image, filename, extraheader)


### This version of lp_write works, but need to check whether to transpose the data or not. Currently it thinks that it is doing a ny, nx flip to nx, ny, but it is not!

In [ ]:
import numpy as np
import struct

def make_lp_header(array, nt=None, t_start=None, delta_t=None):
    """
    Create a standard header for saving La Palma data.
    
    Parameters:
        array (numpy.ndarray): The data array.
        nt (int, optional): Number of time steps.
        t_start (str, optional): Start time.
        delta_t (str, optional): Time interval.
    
    Returns:
        str: The header string.
    """
    # Check image dimensions
    sZ = array.shape
    dims = len(sZ)
    
    if dims < 2:
        raise ValueError("Only 2D or 3D files are supported")
    
    datatype_map = {
        np.uint8: 1,
        np.int16: 2,
        np.int32: 3,
        np.float32: 4
    }
    
    datatype = datatype_map.get(array.dtype.type, 4)
    if nt is not None:
        dims = 3
    if dims == 2:
        ny, nx = sZ
        nt = 1
    else:
        nt, ny, nx = sZ
    
    header = f"datatype={datatype}, dims={dims}, nx={nx}, ny={ny}, nt={nt}"
    
    endianstr = 'l' if struct.unpack('<I', struct.pack('=I', 1))[0] == 1 else 'b'
    header += f", endian={endianstr}"
    
    if t_start is not None:
        header += f", t_start={t_start}"
    if delta_t is not None:
        header += f", delta_t={delta_t}" if isinstance(delta_t, str) else f", delta_t={str(delta_t)}"
    
    return header

def lp_write(image, filename, extraheader='', order='nt nx ny'):
    """
    Writes La Palma data to a file with a specified header.
    
    Parameters:
        image (numpy.ndarray): The image data to write.
        filename (str): The file to write the data to.
        extraheader (str, optional): Additional header information.
        order (str, optional): Order of the input data dimensions.
                               Default is 'nt nx ny'. Other option is 'nt ny nx'.
    """
    # Check image dimensions
    sZ = image.shape
    dims = len(sZ)
    
    if dims < 2:
        raise ValueError("Only 2D or 3D files are supported")
    
    if dims == 2:
        # Convert 2D array to 3D array with 1 timestep
        image = image[np.newaxis, :, :]
        sZ = image.shape
        dims = len(sZ)
    
    if order == 'nt ny nx':
        # Reorder dimensions to (nt, nx, ny)
        image = image.transpose((0, 2, 1))
    elif order != 'nt nx ny':
        raise ValueError("Invalid order specified. Use 'nt nx ny' or 'nt ny nx'.")
    
    sZ = image.shape
    nt, nx, ny = sZ
    datatype = image.dtype
    
    # Create the header
    bheader = make_lp_header(image, nt)
    header = extraheader + ' : ' + bheader
    
    with open(filename, 'wb') as f:
        # Write the header (first 512 bytes)
        header_bytes = header.encode('ascii')
        header_bytes_padded = header_bytes.ljust(512, b'\x00')
        f.write(header_bytes_padded)
        
        # Convert image to float32 if the datatype is not supported
        if datatype not in [np.uint8, np.int16, np.int32, np.float32]:
            image = image.astype(np.float32)
            datatype = image.dtype

        # Write the image data
        f.write(image.tobytes())

# Example usage
# image = np.random.rand(10, 100, 100).astype(np.float32)  # Example 3D image (nt, ny, nx)
# filename = 'example.lp'
# extraheader = 'Additional Header Information'
# lp_write(image, filename, extraheader, order='nt ny nx')


In [ ]:
bazi_im = idl_model_im[2]
blos_im = idl_model_im[-2]
bhor_im = idl_model_im[-1]
blos_im.shape

In [ ]:
filename = 'temp/test_blos.fcube'
extraheader = 'Additional Header Information'
lp_write(blos_im, filename, extraheader, order='nt nx ny') # it is still confusing between nx, ny order. need to think on this and fix it later.

In [ ]:
# reshape model_im[0] to (1, nx, ny) 
bazi_im = model_im[2].reshape(ny,1, 1, nx)
blos_im = model_im[-2].reshape(ny,1, 1, nx)
bhor_im = model_im[-1].reshape(ny,1, 1, nx)

In [ ]:
type(blos_im)

In [ ]:
blos_im.shape

In [ ]:
# lp.writeto('Bazi_6173_2020-08-07_T083019.fcube', bazi_im, dtype='float32')
# lp.writeto('Blos_6173_2020-08-07_T083019.fcube', blos_im, dtype='float32')
# lp.writeto('Bhor_6173_2020-08-07_T083019.fcube', bhor_im, dtype='float32')

In [ ]:
write(blos_im, 'New_Blos_6173_2020-08-07_T083019', stokes=False, sp=False, path='temp/')

In [ ]:
writeto('temp/Bazi_6173_2020-08-07_T083019.fcube', model_im[2], dtype='float32')

In [ ]:
def writeto(filename, image, extraheader='', dtype=None, verbose=False,
            append=False):

In [ ]:
model_im[2].shape

In [ ]:
ff = iu.load_fits_data(out_file_name)

In [ ]:
iu.plot_image(ff)

In [ ]:
hh = iu.load_fits_header('temp/inv_mos.fits')

#### Things to complete
- [x] Move all the inputs to a dictionary and later save them in the header of the output file. Also add the best seeing frame number.
- [x] Move the preprocessing steps like plotting and FOV details as an optional but default true step
- [x] Plot a rectangle to show cropping region is true
- [ ] Save fits with [blos, theta, phi, vlos + errors + mask] for each frame (temporarily) and later combine for final fits
- [ ] Check for option to convert to fcube and icube formats using ispy or helita tools
- [ ] Add option to do only one frame separately if user wants.
- [ ] Add fov angle and other inputs needed for ambiguity resolution and remap in header

#### To do for final cube
- [x] Pick the best seeing frame from the dataset
- [x] Run the full inversion for the best seeing frame
- [ ] Use this output as an initial guess for the other frames
